In [1]:
%cd ..

/home/ubuntu/ppc-experiments-germany/ppc_experiments


In [2]:
import argparse
import collections
import lightning
import numpy as np
import pyro
import torch
import logger, train

/home/ubuntu/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
Seed set to 123


In [3]:
# pyro.enable_validation(True)
# torch.autograd.set_detect_anomaly(True)

In [4]:
config, (data, model, trainer) = train.from_file("experiments/ppc_minibmnist_config.json")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [5]:
trainer.fit(model, data, ckpt_path="saved/models/MiniBouncingMnist_Ppc/0515_052016/checkpoint_979.ckpt")

You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable

ProcessExitedException: process 1 terminated with signal SIGSEGV

In [ ]:
model.graph.clear()
model.eval()
model.cpu()
model.particles['valid'].cpu()

In [ ]:
num_valid_data = len(data.val_dataloader().dataset) % data.val_dataloader().batch_size

In [ ]:
model._load_particles(torch.tensor(list(range(num_valid_data))), False)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
xs, target, index = list(data.val_dataloader())[0]

In [ ]:
xs.shape

In [ ]:
for t in range(xs.shape[1]):
    plt.imshow(xs[0, t, :, :])
    plt.show()

In [ ]:
for site in model.graph.nodes:
    model.graph.nodes[site]['is_observed'] = site in model.particles['valid']

In [ ]:
predictive = pyro.condition(model.graph, data={k: v['value'] for k, v in model.graph.nodes.items()})
estimates = predictive(xs=xs, P=model.num_particles, prior=True)

In [ ]:
estimates = torch.stack(estimates, dim=-3)

In [ ]:
estimate = estimates.mean(dim=0)[0]

In [ ]:
for t in range(estimate.shape[0]):
    plt.imshow(estimate[t, :, :].detach().numpy())
    plt.show()